# WML Federated Learning MNIST Admin Example

This notebook is intended to be run by the Federated Learning experiment administrator.

### Pre-req

The user must upload a PMML model in zip format to the same project where you are running this notebook. In your CPD project, click **Add to project < Model from file** and upload the tf_mnist_model.zip to the same project where you are running this aggregator notebook. 

### Define Variables

**Note:** You need to fill in the credentials for the notebook to run properly.

In [1]:
API_VERSION = "2019-10-25"

# hostname of CP4D cluster
CP4D_HOST = # <host URL>
CP4D_URL = "https://" + CP4D_HOST

# CP4D user
WS_USER = # <username>
WS_PASSWORD = # <password>
# user id for admin user, get this in Administration < User Management (CP4D_URL/zen-admin/?deployment_target=icp4data#/usermgmt-ui)
WS_USERID = # <user ID>

# project id where this notebook resides and model id you uploaded earlier. 
PROJECT_ID = "" # You can get this by looking in your project URL
MODEL_ID = "" # After you upload the model, click on it to view details and you can get this by looking in your model's in the URL

### Import Libraries

In [2]:
import urllib3
import requests
import json
from string import Template

urllib3.disable_warnings()

### Obtain CP4D Authentication Token

In [ ]:
import base64

ws_userpass = WS_USER + ":" + WS_PASSWORD
enc_bytes = base64.b64encode(ws_userpass.encode("utf-8"))
ws_auth = str(enc_bytes, "utf-8")

token_resp = requests.get(CP4D_URL + "/v1/preauth/validateAuth",
                          headers={"Content-Type": "application/json",
                                   "Authorization": "Basic " + ws_auth},
                          verify=False)

print("Token response: " + token_resp.content.decode("utf-8"))
token = "Bearer " + json.loads(token_resp.content.decode("utf-8"))["accessToken"]
print("WS Token: %s " % token)

### Create Remote Training System

In [4]:
wml_remote_training_system_asset_one_def = Template("""
{
  "name": "Remote Party 1",
  "project_id": "$projectId",
  "description": "Sample Remote Training System",
  "tags": [ "Federated Learning" ],
  "organization": {
    "name": "IBM",
    "region": "US"
  },
  "allowed_identities": [
    {
      "id": "$userID",
      "type": "user"
    }
  ],
  "remote_admin": {
    "id": "$userID",
    "type": "user"
  }
}
""").substitute(userID = WS_USERID,
                projectId = PROJECT_ID)


wml_remote_training_system_one_resp = requests.post(CP4D_URL + "/ml/v4/remote_training_systems", 
                                                    headers={"Content-Type": "application/json",
                                                             "Authorization": token}, 
                                                    params={"version": API_VERSION,
                                                            "project_id": PROJECT_ID}, 
                                                    data=wml_remote_training_system_asset_one_def, 
                                                    verify=False)

print(wml_remote_training_system_one_resp)
status_json = json.loads(wml_remote_training_system_one_resp.content.decode("utf-8"))
print("Create wml_remote_training_system_one asset response: %s"  % json.dumps(status_json, indent=4))
wml_remote_training_system_one_asset_uid = status_json["metadata"]["id"]
print("WML wml_remote_training_system_one asset uid: %s" % wml_remote_training_system_one_asset_uid)

<Response [201]>
Create wml_remote_training_system_one asset response: {
    "entity": {
        "allowed_identities": [
            {
                "id": "1000330999",
                "type": "user"
            }
        ],
        "organization": {
            "name": "IBM",
            "region": "US"
        },
        "remote_admin": {
            "id": "1000330999",
            "type": "user"
        }
    },
    "metadata": {
        "created_at": "2020-11-17T04:07:32.789Z",
        "description": "Sample Remote Training System",
        "id": "327a5c10-42c0-4136-8342-ca055581ab65",
        "modified_at": "2020-11-17T04:07:32.789Z",
        "name": "Remote Party 1",
        "owner": "1000330999",
        "project_id": "aa9d16a0-91c4-4992-93e0-cd0f96e077ca",
        "tags": [
            "Federated Learning"
        ]
    }
}
WML wml_remote_training_system_one asset uid: 327a5c10-42c0-4136-8342-ca055581ab65


### Create FL Training Job

In [5]:
training_payload = Template(""" 
{
  "name": "FL Aggregator",
  "tags": [
    {
      "value": "tags_jobs_fl",
      "description": "Sample FL Aggregator"
    }
  ],
  "federated_learning": {
    "model": {
      "spec": {
        "id": "$modelID"
      },
      "type": "tensorflow"
    },
    "fusion_type": "iter_avg",
    "rounds": 5,
    "remote_training" : {
      "quorum": 1.0,
      "remote_training_systems": [ { "id" : "$rts_one", "required" : true  } ]
    },
    "hardware_spec": {
      "name": "XS"
    }
  },
  "training_data_references": [],
  "results_reference": {
    "type": "fs",
    "name": "outputData",
    "location": {
      "path": "projects/$projectId"
    }
  },
  "project_id": "$projectId"  
}
""").substitute(modelID = MODEL_ID,
                projectId = PROJECT_ID,
                rts_one = wml_remote_training_system_one_asset_uid)

print("Training payload: %s" % training_payload)

training_launch_resp = requests.post(CP4D_URL + "/ml/v4/trainings", 
                                     headers={"Content-Type": "application/json",
                                              "Authorization": token},
                                     data=training_payload,
                                     verify=False)

print(training_launch_resp)
print("Launch training response %s"  % training_launch_resp.content.decode("utf-8"))
training_id = json.loads(training_launch_resp.content.decode("utf-8"))["metadata"]["id"]
print("Training ID: %s" % training_id)

Training payload:  
{
  "name": "FL Aggregator",
  "tags": [
    {
      "value": "tags_jobs_fl",
      "description": "Sample FL Aggregator"
    }
  ],
  "federated_learning": {
    "model": {
      "spec": {
        "id": "3f00a787-0bfe-444d-8d3c-ba82ee11e34e"
      },
      "type": "tensorflow"
    },
    "fusion_type": "iter_avg",
    "rounds": 5,
    "remote_training" : {
      "quorum": 1.0,
      "remote_training_systems": [ { "id" : "327a5c10-42c0-4136-8342-ca055581ab65", "required" : true  } ]
    },
    "hardware_spec": {
      "name": "XS"
    }
  },
  "training_data_references": [],
  "results_reference": {
    "type": "fs",
    "name": "outputData",
    "location": {
      "path": "projects/aa9d16a0-91c4-4992-93e0-cd0f96e077ca"
    }
  },
  "project_id": "aa9d16a0-91c4-4992-93e0-cd0f96e077ca"  
}

<Response [201]>
Launch training response {"metadata":{"created_at":"2020-11-17T04:07:43.558Z","guid":"bf53d601-b116-4a77-9735-b40350eba49e","id":"bf53d601-b116-4a77-9735-b40350e

### Get Training Job Status


In [12]:
status_full = requests.get(CP4D_URL + "/ml/v4/trainings/" + training_id,
                           headers={"Content-Type": "application/json",
                                    "Authorization": token},
                           params={"version": API_VERSION,
                                   "project_id": PROJECT_ID},
                           verify=False)

print(status_full)
status_json = json.loads(status_full.content.decode("utf-8"))
print("Full training job status: "+ json.dumps(status_json, indent=4))

<Response [200]>
Full training job status: {
    "metadata": {
        "created_at": "2020-11-17T04:07:43.558Z",
        "guid": "bf53d601-b116-4a77-9735-b40350eba49e",
        "id": "bf53d601-b116-4a77-9735-b40350eba49e",
        "modified_at": "2020-11-17T04:18:24.012Z",
        "name": "FL Aggregator",
        "project_id": "aa9d16a0-91c4-4992-93e0-cd0f96e077ca",
        "tags": [
            "tags_jobs_fl"
        ]
    },
    "entity": {
        "federated_learning": {
            "fusion_type": "iter_avg",
            "hardware_spec": {
                "name": "XS"
            },
            "model": {
                "spec": {
                    "id": "3f00a787-0bfe-444d-8d3c-ba82ee11e34e"
                },
                "type": "tensorflow"
            },
            "remote_training": {
                "quorum": 1.0,
                "remote_training_systems": [
                    {
                        "id": "327a5c10-42c0-4136-8342-ca055581ab65",
                     

### Wait For Aggregator Status To Become Accepting Parties

### Get Variables And Paste Into Party Notebook

In [ ]:
print("CP4D_HOST = \"%s\"" % CP4D_HOST)
print("CP4D_URL = \"%s\"" % CP4D_URL)
print("WS_USER = \"%s\"" % WS_USER)
print("WS_PASSWORD = \"%s\"" % WS_PASSWORD)
print("TRAINING_ID = \"%s\"" % training_id)
print("RTS_ID = \"%s\"" % wml_remote_training_system_one_asset_uid)

# <hr>
Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.
 <br><br>
<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>